<a href="https://colab.research.google.com/github/Daniel-R-Armstrong/Alice-s-Restaurant/blob/master/Project_scraping_loopnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from lxml.html import fromstring #get_proxies
from itertools import cycle # get_proxies
import traceback #get_proxies
import datetime
import csv

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/'My Drive'/data/loopnet

/content/drive/My Drive/data/loopnet


In [0]:
ls

60007.csv  60133.csv  60174.csv  60190.csv  60504.csv  60527.csv  60564.csv
60101.csv  60137.csv  60181.csv  60191.csv  60514.csv  60532.csv  60565.csv
60103.csv  60139.csv  60184.csv  60199.csv  60515.csv  60540.csv  60599.csv
60106.csv  60143.csv  60185.csv  60439.csv  60516.csv  60555.csv
60108.csv  60148.csv  60187.csv  60440.csv  60517.csv  60559.csv
60120.csv  60157.csv  60188.csv  60502.csv  60521.csv  60561.csv
60126.csv  60172.csv  60189.csv  60503.csv  60523.csv  60563.csv


# scrape

## get free proxies

In [0]:
#https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/

from lxml.html import fromstring
import requests
from itertools import cycle
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    headers = {'User-Agent': user_agent}
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

url = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    #print("Request #%d"%i)
    print(f"Request #{i}.")
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy},headers=headers)
        print(response.json())
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")

Request #1.
Skipping. Connnection error
Request #2.
Skipping. Connnection error
Request #3.
Skipping. Connnection error
Request #4.
Skipping. Connnection error
Request #5.
Skipping. Connnection error
Request #6.
Skipping. Connnection error
Request #7.
Skipping. Connnection error
Request #8.
Skipping. Connnection error
Request #9.
Skipping. Connnection error
Request #10.
Skipping. Connnection error


In [0]:

print(proxies)

## To do-not started

In [0]:
# I would like to get more data out of this website

# https://www.zipdatamaps.com/60185 #60185 is zip

# LoopNet scrape

## functions for loopnet scrape

In [0]:
# USED IN PROPERTIES TABLE
def title_find(tag):
  try:
    title = tag.find('span', class_="listingTitle").text
    if title == None:
      return 'NA'
    else:
      #print(title)
      return title
  except Exceptions as e:
    print("issue with Title: " + e)
    return 'Title error'

In [0]:
# USED IN PROPERTIES TABLE
def href_find(tag):
  try:
    href = tag.find('div', class_="listingDescription").a.attrs['href']
    if href ==None:
      return 'NA'
    else:
      #print("Href:"+ href)
      return href
  except EXception as e:
    print("issue with href: " + e)
    return 'href Error'

In [0]:
# USED IN PROPERTIES TABLE
def desc_find(tag):
  try:
    desc = tag.find('span', class_="propertyDescription").text
    if desc==None:
      return 'NA'
    else:
      #print("Property Description: " + desc)
      return desc
  except Exception as e:
    Print("Issue with Description: " + e)
    return 'Description Error'

In [0]:
# USED IN PROPERTIES TABLE
def img_find(tag):
  try:
    img = tag.find('div',class_='listingPhoto').img['src']
    if img == None:
      return 'NA'
    else:
      #Print("Image: "+ img)
      return img
  except Exeption as e:
    print("issue with Image: " + e)
    return 'Image Error'
    

In [0]:
def price_find(tag):
  try:
    price = tag.find('td', string = re.compile('Price'))
    if price == None:
      #print("Price: Not Disclosed")
      return 'Not Disclosed'
    else:
      #print("prince: "+ price.next_sibling.next_sibling.text)
      return price.next_sibling.next_sibling.text
  except Exception as e:
    print("issue with price: " + e)
    return 'Price Error'

In [0]:
def ptype_find(tag):
  ptype = tag.find('td', string = re.compile('Property Type'))
  if ptype == None:
    #print("Property Type: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Property Type: " + ptype.next_sibling.next_sibling.text )
    return ptype.next_sibling.next_sibling.text

In [0]:
def stype_find(tag):
  stype = tag.find('td', string = re.compile('Sub-Type'))
  if stype == None:
    #print("Sub-Type: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Sub-Type: "+ stype.next_sibling.next_sibling.text )
    return stype.next_sibling.next_sibling.text

In [0]:
def spaces_find(tag):
  spaces = tag.find('td', string = re.compile('Spaces'))
  if spaces == None:
    #print("Spaces: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Spaces: "+ spaces.next_sibling.next_sibling.text)
    return spaces.next_sibling.next_sibling.text 

In [0]:
def size_find(tag):
  size = tag.find('td', string = re.compile('Building Size'))
  if size == None:
    #print("Buliding Size: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Building Size: "+ size.next_sibling.next_sibling.text)
    return size.next_sibling.next_sibling.text  

In [0]:
def space_find(tag):
  space = tag.find('td', string = re.compile('Space Available'))
  if space == None:
    #print("Space Available: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Space Available: "+ space.next_sibling.next_sibling.text)
    return space.next_sibling.next_sibling.text 

In [0]:
def cap_find(tag):
  cap = tag.find('td', string = re.compile('Cap Rate'))
  if cap == None:
    #print("Cap Rate: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Cap Rate: "+ cap.next_sibling.next_sibling.text)
    return cap.next_sibling.next_sibling.text

In [0]:
def class_find(tag):
  bclass = tag.find('td', string = re.compile('Building Class'))
  if bclass == None:
    #print("Building Class: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Building Class: "+ bclass.next_sibling.next_sibling.text)
    return bclass.next_sibling.next_sibling.text

In [0]:
def lot_find(tag):
  lot = tag.find('td', string = re.compile('Lot Size'))
  if lot == None:
    #print("Lot Size: Not Disclosed")
    return 'Not Disclosed'
  else:
    #print("Lot Size: " + lot.next_sibling.next_sibling.text)
    return lot.next_sibling.next_sibling.text

In [0]:
def pg_limit(tag):
  try:
    page_nums = tag.find('div', class_ = 'searchPaging').get_text()
    borderless = tag.find('a', class_ = 'searchPagingBorderless').get_text()
    result = re.sub('[^0-9.]','', page_nums)
    dots = result.find('...')
    if dots > 0:
        return int(result[dots+3:])
    else:
        return int(borderless)
        
  except Exception as e:
      print (e)
      return int(1)

In [0]:
results = []
def properties_table_maker(soup):
  page = soup.find_all('div', {re.compile('listingContainer*')})
                               
  for con in page:
    listing = {}
    listing['Property Name'] = title_find(con)
    listing['Listing URL'] = href_find(con)
    listing['Short Description']= desc_find(con)
    listing['Listing Image']= img_find(con)
    listing['Price']= price_find(con)
    listing['Property Type']= ptype_find(con)
    listing['Sub-Type']= stype_find(con)
    listing['Spaces']= spaces_find(con)
    listing['Building Size']= size_find(con)
    listing['Space Available']= space_find(con)
    listing['Cap Rate']= cap_find(con)
    listing['Building Class']= class_find(con)
    listing['Lot Size']= lot_find(con)
    listing['Date checked']= datetime.datetime.now(datetime.timezone.utc)
    results.append(listing)

In [0]:
#todo get headers
#todo buy and use proxie list
from warnings import warn # warning about response(status code not 200)
from time import time # time functions
from time import sleep # sleeping functions
from random import randint # for random integer used in sleep
import requests
from IPython.core.display import clear_output #for clear output

#proxies = {'http':'https://45.229.104.185:33903'}
#url = 'https://httpbin.org/user-agent' # tests your user agent
#url = 'https://httpbin.org/ip' # test proxies
#url = 'https://ipinfo.io #test ip address too
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
headers = {'User-Agent': user_agent}


def listings_scrape(zipcode):
  url_one = 'https://www.loopnet.com/zip/' + zipcode + '-commercial-real-estate/'
  #page_position = url_one.index('')
  start_time = time()
  num_requests = 0
  try:
    results =[]
    page_one = requests.get(url_one,headers=headers)# add proxies=proxies
    page_one_soup = BeautifulSoup(page_one.content.decode("utf-8"),"html.parser")# pick a parser['lxml', 'html.parser', 'html5lib', 'xml']
    print("there is "+ str(pg_limit(page_one_soup))+" pages of listings for zipcode: "+ zipcode)
    properties_table_maker(page_one_soup)
    sleep(randint(8,15))
    num_requests += 1
    elapsed_time = time() - start_time
    file_name = zipcode + ".csv"
    frequency = num_requests/elapsed_time
    print('Request:{}; Frequency: {} num_requests/s'.format(num_requests, frequency))
    clear_output(wait = True)
    
    #pages iterator
    for i in range(2, pg_limit(page_one_soup) + 1):
      i_str = str(i)
      url = url_one + i_str + '/'
      page = requests.get(url, headers=headers)# add proxies=proxies)
      page_soup = BeautifulSoup(page.content.decode("utf-8"), "html.parser")
      properties_table_maker(page_soup)
      
      #sleep and track
      sleep(randint(4,10)) #8,15 worked for dupage on 06/23/2019
      num_requests += 1
      elapsed_time = time() - start_time
      print('Request:{}; Frequency: {} num_requests/s'.format(num_requests, frequency))
      clear_output(wait = True)
      
      # Throw a warning for non-200 status codes
      if page.status_code != 200:
        warn('Request: {}; Status code: {}'.format(num_requests, page.status_code))
      
      # Break the loop if the number of requests is greater than expected
      if num_requests > 72:
        warn('Number of requests was greater than expected.')
        break 
        
    keys = results[0].keys()
    with open(file_name, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)
    return results
      #return spaces_df.to_csv(zipcode + '.csv')
        
  except Exception as e:
    print(e)
    return None

##Loopnet by zipcode scrape

##zips by county

In [0]:
cook_zips=['60004','60005','60007','60008','60016','60018',
           '60022','60025','60026','60029','60043','60053',
           '60056','60062','60067','60068','60070','60074',
           '60076','60077','60090','60091','60093','60104',
           '60107','60130','60131','60133','60141','60153',
           '60154','60155','60160','60162','60163','60164',
           '60165','60168','60169','60171','60173','60176',
           '60192','60193','60194','60195','60201','60202',
           '60203','60301','60302','60304','60305','60402',
           '60406','60409','60411','60415','60419','60422',
           '60425','60426','60428','60429','60430','60438',
           '60439','60443','60445','60452','60453','60455',
           '60456','60457','60458','60459','60461','60462',
           '60463','60464','60465','60466','60467','60469',
           '60471','60472','60473','60475','60476','60477',
           '60478','60480','60482','60483','60487','60501',
           '60513','60525','60526','60534','60546','60558',
           '60601','60602','60603','60604','60605','60606',
           '60607','60608','60609','60610','60611','60612',
           '60613','60614','60615','60616','60617','60618',
           '60619','60620','60621','60622','60623','60624',
           '60625','60626','60628','60629','60630','60631',
           '60632','60633','60634','60636','60637','60638',
           '60639','60640','60641','60643','60644','60645',
           '60646','60647','60649','60651','60652','60653',
           '60654','60655','60656','60657','60659','60660',
           '60661','60706','60707','60712','60714','60803',
           '60804','60805','60827']



In [0]:
dupage_zips=['60101','60103','60106','60108','60126', 
             '60137','60139','60143','60148','60157', 
             '60172','60181','60184','60185','60187', 
             '60188','60189','60190','60191','60502', 
             '60504','60514','60515','60516','60517', 
             '60521','60523','60527','60532','60540',
             '60555','60559','60561','60563','60565']

## Functions

In [0]:
# USED IN PROPERTIES TABLE
def title_find(tag):
    try:
        title = tag.find('span', class_="listingTitle").text
        if title == None:
            return 'NA'
        else:
            #print(title)
            return title
    except Exceptions as e:
        print("issue with Title: " + e)
        return 'Title error'
    
def href_find(tag):
    try:
        href = tag.find('div', class_="listingDescription").a.attrs['href']
        if href ==None:
            return 'NA'
        else:
            #print("Href:"+ href)
            return href
    except EXception as e:
        print("issue with href: " + e)
        return 'href Error'
    
def desc_find(tag):
    try:
        desc = tag.find('span', class_="propertyDescription").text
        if desc==None:
            return 'NA'
        else:
            #print("Property Description: " + desc)
            return desc
    except Exception as e:
        Print("Issue with Description: " + e)
        return 'Description Error'
    
def img_find(tag):
    try:
        img = tag.find('div',class_='listingPhoto').img['src']
        if img == None:
            return 'NA'
        else:
            #Print("Image: "+ img)
            return img
    except Exeption as e:
        print("issue with Image: " + e)
        return 'Image Error'
    
def price_find(tag):
    try:
        price = tag.find('td', string = re.compile('Price'))
        if price == None:
            #print("Price: Not Disclosed")
            return 'Not Disclosed'
        else:
            #print("prince: "+ price.next_sibling.next_sibling.text)
            return price.next_sibling.next_sibling.text
    except Exception as e:
        print("issue with price: " + e)
        return 'Price Error'
    
def ptype_find(tag):
    ptype = tag.find('td', string = re.compile('Property Type'))
    if ptype == None:
        #print("Property Type: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Property Type: " + ptype.next_sibling.next_sibling.text )
        return ptype.next_sibling.next_sibling.text
    
def stype_find(tag):
    stype = tag.find('td', string = re.compile('Sub-Type'))
    if stype == None:
        #print("Sub-Type: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Sub-Type: "+ stype.next_sibling.next_sibling.text )
        return stype.next_sibling.next_sibling.text
    
def spaces_find(tag):
    spaces = tag.find('td', string = re.compile('Spaces'))
    if spaces == None:
        #print("Spaces: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Spaces: "+ spaces.next_sibling.next_sibling.text)
        return spaces.next_sibling.next_sibling.text
    
def size_find(tag):
    size = tag.find('td', string = re.compile('Building Size'))
    if size == None:
        #print("Buliding Size: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Building Size: "+ size.next_sibling.next_sibling.text)
        return size.next_sibling.next_sibling.text    
    
def space_find(tag):
    space = tag.find('td', string = re.compile('Space Available'))
    if space == None:
        #print("Space Available: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Space Available: "+ space.next_sibling.next_sibling.text)
        return space.next_sibling.next_sibling.text     

def cap_find(tag):
    cap = tag.find('td', string = re.compile('Cap Rate'))
    if cap == None:
        #print("Cap Rate: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Cap Rate: "+ cap.next_sibling.next_sibling.text)
        return cap.next_sibling.next_sibling.text    
    
def class_find(tag):
    bclass = tag.find('td', string = re.compile('Building Class'))
    if bclass == None:
        #print("Building Class: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Building Class: "+ bclass.next_sibling.next_sibling.text)
        return bclass.next_sibling.next_sibling.text
    
def lot_find(tag):
    lot = tag.find('td', string = re.compile('Lot Size'))
    if lot == None:
        #print("Lot Size: Not Disclosed")
        return 'Not Disclosed'
    else:
        #print("Lot Size: " + lot.next_sibling.next_sibling.text)
        return lot.next_sibling.next_sibling.text
    
def pg_limit(tag):
    try:
        page_nums = tag.find('div', class_ = 'searchPaging').get_text()
        borderless = tag.find('a', class_ = 'searchPagingBorderless').get_text()
        result = re.sub('[^0-9.]','', page_nums)
        dots = result.find('...')
        if dots > 0:
            return int(result[dots+3:])
        else:
            return int(borderless)
        
    except Exception as e:
        print (e)
        return int(1)    
    
def properties_table_maker(soup):
    page = soup.find_all('div', {re.compile('listingContainer*')})
    # results = []                           
    for con in page:
        listing = {}
        listing['Property Name'] = title_find(con)
        listing['Listing URL'] = href_find(con)
        listing['Short Description']= desc_find(con)
        listing['Listing Image']= img_find(con)
        listing['Price']= price_find(con)
        listing['Property Type']= ptype_find(con)
        listing['Sub-Type']= stype_find(con)
        listing['Spaces']= spaces_find(con)
        listing['Building Size']= size_find(con)
        listing['Space Available']= space_find(con)
        listing['Cap Rate']= cap_find(con)
        listing['Building Class']= class_find(con)
        listing['Lot Size']= lot_find(con)
        listing['Date checked']= datetime.datetime.now(datetime.timezone.utc)
        results.append(listing)    

In [0]:
#todo get headers
#todo buy and use proxie list
from warnings import warn # warning about response(status code not 200)
from time import time # time functions
from time import sleep # sleeping functions
from random import randint # for random integer used in sleep
import requests
from IPython.core.display import clear_output #for clear output

#proxies = {'http':'https://45.229.104.185:33903'}
#url = 'https://httpbin.org/user-agent' # tests your user agent
#url = 'https://httpbin.org/ip' # test proxies
#url = 'https://ipinfo.io #test ip address too
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
headers = {'User-Agent': user_agent}
results =[]

def listings_scrape(zipcode):
    url_one = 'https://www.loopnet.com/zip/' + zipcode + '-commercial-real-estate/'
    #page_position = url_one.index('')
    start_time = time()
    num_requests = 0
    results.clear()
    try:
        page_one = requests.get(url_one,headers=headers)# add proxies=proxies
        page_one_soup = BeautifulSoup(page_one.content.decode("utf-8"),"html.parser")# pick a parser['lxml', 'html.parser', 'html5lib', 'xml']
        print("there is "+ str(pg_limit(page_one_soup))+" pages of listings for zipcode: "+ zipcode)
        properties_table_maker(page_one_soup)
        num_requests += 1
        elapsed_time = time() - start_time
        file_name = zipcode + ".csv"
        frequency = num_requests/elapsed_time
        print('zipcode: {}; Request:{}; Frequency: {} num_requests/s'.format(zipcode, num_requests, frequency))
        clear_output(wait = True)
    
        # pages iterator
        for i in range(2, pg_limit(page_one_soup) + 1):
            i_str = str(i)
            url = url_one + i_str + '/'
            page = requests.get(url, headers=headers)# add proxies=proxies)
            page_soup = BeautifulSoup(page.content.decode("utf-8"), "html.parser")
            properties_table_maker(page_soup)
            # sleep and track
            sleep(randint(1,3)) #8,15 worked for dupage on 06/23/2019
            num_requests += 1
            elapsed_time = time() - start_time
            print('Request:{}; Frequency: {} num_requests/s'.format(num_requests, frequency))
            clear_output(wait = True)
            # Throw a warning for non-200 status codes
            if page.status_code != 200:
                warn('Request: {}; Status code: {}'.format(num_requests, page.status_code))
            # Break the loop if the number of requests is greater than expected
            if num_requests > 72:
                warn('Number of requests was greater than expected.')
                break 
            
        keys = results[0].keys()
        with open(file_name, 'w') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(results)
    
        # return spaces_df.to_csv(zipcode + '.csv')
        
    except Exception as e:
        print(e)
        return None

In [0]:
for i in dupage_zips:
    listings_scrape(i)


'NoneType' object has no attribute 'get_text'


In [0]:
for i in cook_zips:
    listings_scrape(i)


'NoneType' object has no attribute 'get_text'


In [0]:
# results =[]
# listings_scrape('60199')

In [0]:
ls


60101.csv  60137.csv  60181.csv  60189.csv  60514.csv  60523.csv  60559.csv
60103.csv  60139.csv  60184.csv  60190.csv  60515.csv  60527.csv  60561.csv
60106.csv  60143.csv  60185.csv  60191.csv  60516.csv  60532.csv  60563.csv
60108.csv  60148.csv  60187.csv  60502.csv  60517.csv  60540.csv  60565.csv
60126.csv  60172.csv  60188.csv  60504.csv  60521.csv  60555.csv


Check the length of each of the csv files
```
! wc -l *.csv 
```

In [0]:
! wc -l *.csv 

    165 60004.csv
     98 60005.csv
    283 60007.csv
    113 60008.csv
     74 60016.csv
    158 60018.csv
     14 60022.csv
    102 60025.csv
     39 60026.csv
      8 60043.csv
     67 60053.csv
    120 60056.csv
    258 60062.csv
    150 60067.csv
     75 60068.csv
     27 60070.csv
    105 60074.csv
    128 60076.csv
    111 60077.csv
    165 60090.csv
     32 60091.csv
     60 60093.csv
    151 60101.csv
    108 60103.csv
     29 60104.csv
    142 60106.csv
     95 60107.csv
     74 60108.csv
    147 60126.csv
     39 60130.csv
    156 60131.csv
     55 60133.csv
     85 60137.csv
     60 60139.csv
     97 60143.csv
    206 60148.csv
     38 60153.csv
     46 60154.csv
     37 60155.csv
    141 60160.csv
     42 60162.csv
      2 60163.csv
     46 60164.csv
      8 60165.csv
     75 60169.csv
     17 60171.csv
     53 60172.csv
    290 60173.csv
     41 60176.csv
    103 60181.csv
      2 60184.csv
    211 60185.csv
    142 60187.csv
    140 60188.csv
     60 60189.csv
     29 60

In [0]:
c=pd.read_csv("60190.csv")
c['Listing URL'][0]

'/Listing/27W093-27W121-Geneva-Rd-Winfield-IL/11956940/'


https://www.loopnet.com/Listing/27W230-Beecher-Ave-Winfield-IL/9456034/		


Listing ID: 9456034 Date Created: 10/5/2017 Last Updated: 6/10/2019

DESCRIPTION, HIGHLIGHTS, more pictures, SALE NOTES, AMENITIES, UTILITIES